In [ ]:
from typing import List, Optional, Literal,Annotated, TypedDict
from pydantic import BaseModel
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, AnyMessage
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from langgraph.types import interrupt, Command

llm = init_chat_model(model="openai:gpt-4o")

QUESTIONS = [
    "What is the name of the competitor?",
    "Whats your favorite color?",
    "Pick a number between 1 and 100",
    "What is your name?"
]

class CompetitiveAnalysisState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    user_answered_all_questions: bool


In [ ]:
system_prompt = """
You need to verify the user answered all the questions and if not, ask the next question.
The list of questions is:
{QUESTIONS}

User the history of messages to verify if the user answered all the questions.
"""

class next_question(BaseModel):
    question: str
    answered_all_questions: bool

In [ ]:
def CompetitiveAnalysisNode(state: CompetitiveAnalysisState):
    system_message = SystemMessage(content=system_prompt)
    history = state['messages']
    messages = [system_message] + history
    llm_with_structured_output = llm.with_structured_output(next_question)
    ai_message = llm_with_structured_output.invoke(messages)

    user_response = interrupt({
        "question": ai_message,
    })
    messages.append(AIMessage(content=ai_message))
    messages.append(HumanMessage(content=user_response))

    return {"messages": messages}





builder = StateGraph(CompetitiveAnalysisState)

builder.add_node("CompetitiveAnalysisNode", CompetitiveAnalysisNode)
builder.add_edge(START, "CompetitiveAnalysisNode")
builder.add_edge("CompetitiveAnalysisNode", END)

graph = builder.compile()


In [1]:
!pip install -qU langchain-community beautifulsoup4


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.cisco.com/site/us/en/products/security/firewalls/index.html")
docs = loader.load()  # returns list of Documents with page_content + metadata

print(docs[0].page_content)  # first 500 chars









Cisco Secure Firewall Advanced Threat Protection - Cisco






































































Skip to main content
Skip to search
Skip to footer















Cisco.com Worldwide



























Products and Services









Close










Solutions









Close










Support








Close









Learn








Close









Why Cisco








Close












Partners








Close












Trials and demos





How to buy





Partners






EN US








Profile






Log in





Trials and demos
















MENU
CLOSE





How to buy





Partners





Profile






Log in






EN US








Close






Close






Close






Close






Close




































Hello, how can I help?



















Products





Security























        Cisco Secure Firewall
    
Gain advanced threat protection at scale without compromising performance across data center, cloud, campus, and 

In [6]:
pip install html2text


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import Html2TextTransformer

urls = ["https://www.cisco.com/site/us/en/products/security/firewalls/index.html","https://www.ibm.com"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()
transformer = Html2TextTransformer()
docs_transformed = transformer.transform_documents(docs)

print(docs_transformed[0].page_content)

Fetching pages: 100%|##########| 2/2 [00:00<00:00,  2.14it/s]

  * Skip to main content
  * Skip to search
  * Skip to footer

Cisco.com Worldwide

###  Products and Services

Close

###  Solutions

Close

###  Support

Close

###  Learn

Close

###  Why Cisco

Close

###  Partners

Close

Trials and demos

How to buy

Partners

EN US

Profile

Log in

Trials and demos

MENU

CLOSE

How to buy

Partners

Profile

Log in

EN US

Close

Close

Close

Close

Close

__

Hello, how can I help?

  1. Products
  2. Security

#  Cisco Secure Firewall

Gain advanced threat protection at scale without compromising performance
across data center, cloud, campus, and IoT environments, with unified
management across firewalls.

See, try, or buy a firewall  See it in action

Overview Resources Community

See, try, or buy a firewall

## What's new for Cisco Secure Firewall

###  3x better price-performance*

Introducing a new family of advanced firewalls that extends Cisco's Hybrid
Mesh Firewall to distributed branches.

Explore the Secure Firewall 200 Series

##

In [10]:
docs_transformed

[Document(metadata={'source': 'https://www.cisco.com/site/us/en/products/security/firewalls/index.html', 'title': 'Cisco Secure Firewall Advanced Threat Protection - Cisco', 'description': " Cisco's Secure Firewall hardware and software options enhance your security to block more threats and swiftly respond to breaches.", 'language': 'en-US'}, page_content='  * Skip to main content\n  * Skip to search\n  * Skip to footer\n\nCisco.com Worldwide\n\n###  Products and Services\n\nClose\n\n###  Solutions\n\nClose\n\n###  Support\n\nClose\n\n###  Learn\n\nClose\n\n###  Why Cisco\n\nClose\n\n###  Partners\n\nClose\n\nTrials and demos\n\nHow to buy\n\nPartners\n\nEN US\n\nProfile\n\nLog in\n\nTrials and demos\n\nMENU\n\nCLOSE\n\nHow to buy\n\nPartners\n\nProfile\n\nLog in\n\nEN US\n\nClose\n\nClose\n\nClose\n\nClose\n\nClose\n\n__\n\nHello, how can I help?\n\n  1. Products\n  2. Security\n\n#  Cisco Secure Firewall\n\nGain advanced threat protection at scale without compromising performance\na